# Demo mlflow with transformer

First, install pytorch following [here](https://pytorch.org/get-started/locally/). Or just run below block.

In [2]:
!poetry add torch torchvision transformers datasets sentencepiece accelerate

Using version ^2.0.1 for torch
Using version ^0.15.2 for torchvision
Using version ^4.33.1 for transformers
Using version ^2.14.5 for datasets
Using version ^0.1.99 for sentencepiece
Using version ^0.22.0 for accelerate

Updating dependencies
Resolving dependencies... (4.9s)Resolving dependencies... (0.5s)Resolving dependencies... (0.8s)Resolving dependencies... (2.9s)

Package operations: 26 installs, 0 updates, 0 removals

  • Installing frozenlist (1.4.0): Pending...
  • Installing multidict (6.0.4): Pending...
  • Installing multidict (6.0.4): Installing...
  • Installing multidict (6.0.4): Installing...
  • Installing frozenlist (1.4.0): Installing...
  • Installing multidict (6.0.4): Installing...
  • Installing multidict (6.0.4): Installing...
  • Installing frozenlist (1.4.0)
  • Installing multidict (6.0.4): Installing...
  • Installing multidict (6.0.4)
  • Installing aiosignal (1.3.1): Pending...
  • Installing async-timeout (4.0.3): Pending...
  • Installing attrs (23.1.0):

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, AutoTokenizer, DataCollatorWithPadding, Trainer
from datasets import load_dataset
import mlflow


mlflow.autolog()


dataset = load_dataset("rotten_tomatoes")  # doctest: +IGNORE_RESULT
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
training_args = TrainingArguments(
    output_dir="./tmp/",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
)


def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])
dataset = dataset.map(tokenize_dataset, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)  # doctest: +SKIP


trainer.train()